In [23]:
targetFolderHBK = r"H:\Extracted_Features\HBK\HBK_20kHz_original_all_features\features"
targetFolderMCC5 = r"H:\Extracted_Features\MCC5\MCC5_12800Hz_original_all_features_motor_vibration_x\features"
targetFolderSIZA = r"H:\Extracted_Features\SIZA\SIZA_original_all_features\features"
normalization_method = "z_score"

# Environment Setup & Imports

In [24]:
import os
import pandas as pd
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pickle
import numpy as np
import mlflow
from pytorch_lightning.loggers import MLFlowLogger
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score, balanced_accuracy_score, hamming_loss, hinge_loss, jaccard_score, log_loss, precision_score, recall_score, f1_score, make_scorer
from pathlib import Path
from pycaret.classification import * 
from torch import tensor
from torchmetrics.classification import BinaryAccuracy, MulticlassAccuracy
import optuna
import torch
from sklearn.model_selection import train_test_split
from pytorch_tabular import TabularModel
from pytorch_tabular.models import GANDALFConfig, CategoryEmbeddingModel,GatedAdditiveTreeEnsembleConfig, NodeConfig, FTTransformerConfig, TabNetModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    ModelConfig,
    TrainerConfig,
    ExperimentConfig,
)
from collections import Counter
from data_loader import load_feature_data

## Import Dataset

In [25]:
experiment_name = "Best_Hyperparameters_z_score"

In [26]:
df_binary_HBK = load_feature_data(
    features_path=targetFolderHBK,
    include_augmentations=False,      # Only 'original' data
    include_speed_torque=False,       # Drop operating conditions
    binary_classification=True,       # 'healthy' vs 'damaged'
)

Successfully loaded 161 files into a DataFrame with shape (280195, 30)
Applied binary classification: 'healthy' vs 'damaged'.
Dropped 'Speed' and 'Torque' columns.
Final DataFrame shape: (280195, 28)


In [27]:
df_binary_SIZA = load_feature_data(
    features_path=targetFolderMCC5,
    include_augmentations=False,      # Only 'original' data
    include_speed_torque=False,       # Drop operating conditions
    binary_classification=True,       # 'healthy' vs 'damaged'
)

Successfully loaded 36 files into a DataFrame with shape (53928, 30)
Applied binary classification: 'healthy' vs 'damaged'.
Dropped 'Speed' and 'Torque' columns.
Final DataFrame shape: (53928, 28)


In [28]:
df_binary_MCC5 = load_feature_data(
    features_path=targetFolderMCC5,
    include_augmentations=False,      # Only 'original' data
    include_speed_torque=False,       # Drop operating conditions
    binary_classification=True,       # 'healthy' vs 'damaged'
)

Successfully loaded 36 files into a DataFrame with shape (53928, 30)
Applied binary classification: 'healthy' vs 'damaged'.
Dropped 'Speed' and 'Torque' columns.
Final DataFrame shape: (53928, 28)


In [29]:
combined_df = pd.concat([df_binary_HBK, df_binary_SIZA, df_binary_MCC5], ignore_index=True)

In [31]:
normalized_df = normalizeDataframe(combined_df, normalization_method)

In [32]:
features_df_training_normalized, features_df_testing_normalized = train_test_split(
    normalized_df, 
    test_size=0.2,    # e.g., 20% for testing
    random_state=42   # for reproducibility
)

## Helper Functions

In [33]:
def normalizeDataframe(dataframe, normalization_method):
    """
    Normalizes the features of a dataframe using a specified method.

    Args:
        dataframe (pd.DataFrame): The input dataframe with a 'Label' column.
        normalization_method (str): The method to use ("min_max", "z_score", "robust_scaling").

    Returns:
        pd.DataFrame: The dataframe with scaled features.
    """
    # Separate features (X) and the target variable (y)
    y = dataframe['Label']
    X = dataframe.drop(columns=['Label'])

    # Select the scaler based on the chosen method
    if normalization_method == "min_max":
        scaler = MinMaxScaler()
    elif normalization_method == "z_score":
        scaler = StandardScaler()
    elif normalization_method == "robust_scaling":
        scaler = RobustScaler()
    else:
        # Raise an error for an invalid method name
        raise ValueError(f"Unknown normalization_method: '{normalization_method}'")

    # Fit the scaler to the data and transform it
    X_scaled = pd.DataFrame(
        scaler.fit_transform(X),
        columns=X.columns,
        index=X.index
    )

    # Rejoin the scaled features with the label column
    df_scaled = X_scaled.join(y)
    
    return df_scaled

In [34]:
def plotPredictionHistograms(df, domain, normalization):
    # 1) mark correct vs incorrect
    df = df.copy()
    df['prediction_quality'] = np.where(
        df['Label'] == df['prediction_label'],
        'correct',
        'incorrect'
    )
    
    # 2) choose a palette (you can override these colors if you like)
    pal = dict(zip(
        ['correct','incorrect'],
        sns.color_palette(n_colors=2)
    ))
    
    skip = {'Label','prediction_label','prediction_score','prediction_quality'}
    for col in df.columns:
        if col in skip:
            continue
        
        fig, ax = plt.subplots(figsize=(8,4))
        sns.histplot(
            data=df, x=col, hue='prediction_quality',
            palette=pal,
            kde=True, multiple='layer', element='step',
            alpha=0.5,
            ax=ax
        )
        
        # 3) build a manual legend using the same palette
        handles = [
            mpatches.Patch(color=pal[k], label=k)
            for k in ['correct','incorrect']
        ]
        ax.legend(
            handles=handles,
            title='Prediction Quality'
        )
        
        ax.set_title(
            f"Distribution of {col} in the '{domain}' domain\n"
            f"(normalization = '{normalization}')"
        )
        ax.set_xlabel(col)
        ax.set_ylabel("Count")
        plt.tight_layout()
        plt.show()

In [35]:
def get_incorrect_predictions(df):
    return df[
        ((df['Label'] == 'damaged')   & (df['prediction_label'] == 'healthy'))
      | ((df['Label'] == 'healthy')  & (df['prediction_label'] == 'damaged'))
    ].copy()

In [36]:
def get_feature_importance_df(model, df):
    importance = model.feature_importances_
    n = len(importance)
    features = df.columns[:n]
    fi_df = pd.DataFrame({
        'Features': features,
        'importance': importance
    })
    return fi_df.sort_values(by='importance', ascending=False).reset_index(drop=True)

In [37]:
def get_svm_feature_importance_df(model, df):
    if not hasattr(model, 'coef_'):
        raise ValueError("This SVM model has no coefficients. Use a linear kernel.")
    
    importance = model.coef_.ravel()  # Flatten in case of binary classification
    n = len(importance)
    features = df.columns[:n]
    fi_df = pd.DataFrame({
        'Features': features,
        'importance': abs(importance)
    })
    return fi_df.sort_values(by='importance', ascending=False).reset_index(drop=True)


In [40]:
feature_counter = Counter()
def add_top_features(feature_df: pd.DataFrame, top_n: int):
    top_features = feature_df.nlargest(top_n, 'importance')['Features']
    feature_counter.update(top_features)
    
def plot_feature_importance():
    feature_freq = pd.DataFrame(feature_counter.items(), columns=['Feature', 'Count'])
    plt.figure(figsize=(10, 5))
    sns.barplot(data=feature_freq.sort_values(by='Count', ascending=False),
                x='Feature', y='Count')
    plt.xticks(rotation=45)
    plt.title('Feature Frequency Across Experiments')
    plt.tight_layout()
    plt.show()

# Experiment Setup (ML)

## Setup Hyperparameters

In [41]:
experiment = setup(features_df_training_normalized, target='Label', log_experiment = True, experiment_name = experiment_name)

,Description,Value
0,Session id,2003
1,Target,Label
2,Target type,Binary
3,Target mapping,"damaged: 0, healthy: 1"
4,Original data shape,"(310440, 28)"
5,Transformed data shape,"(310440, 28)"
6,Transformed train set shape,"(217308, 28)"
7,Transformed test set shape,"(93132, 28)"
8,Numeric features,27
9,Preprocess,True


2025/09/26 10:27:59 INFO mlflow.tracking.fluent: Experiment with name 'Best_Hyperparameters_z_score' does not exist. Creating a new experiment.


## Add aditional metrics

In [42]:
# Binary classification metrics
add_metric('balanced_acc', 'Balance Acc', balanced_accuracy_score, target='pred', greater_is_better=True)
add_metric('hamming_loss', 'Hamming Loss', hamming_loss, target='pred', greater_is_better=False)
add_metric('jaccard_score', 'Jaccard Score', jaccard_score, target='pred', greater_is_better=True)
add_metric('log_loss', 'Log Loss', log_loss, target='pred_proba', greater_is_better=False)

Name                                                          Log Loss
Display Name                                                  Log Loss
Score Function       <pycaret.internal.metrics.EncodedDecodedLabels...
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: log_loss, dtype: object

In [43]:
all_metrics = get_metrics()
all_metrics

,Name,Display Name,Score Function,Scorer,Target,Args,Greater is Better,Multiclass,Custom
ID,,,,,,,,,
acc,Accuracy,Accuracy,<function accuracy_score at 0x0000029497B4D760>,accuracy,pred,{},True,True,False
auc,AUC,AUC,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(roc_auc_score, response_method=('d...",pred_proba,"{'average': 'weighted', 'multi_class': 'ovr'}",True,True,False
recall,Recall,Recall,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(recall_score, response_method='pre...",pred,{'average': 'weighted'},True,True,False
precision,Precision,Prec.,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(precision_score, response_method='...",pred,{'average': 'weighted'},True,True,False
f1,F1,F1,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(f1_score, response_method='predict...",pred,{'average': 'weighted'},True,True,False
kappa,Kappa,Kappa,<function cohen_kappa_score at 0x0000029497B4D...,"make_scorer(cohen_kappa_score, response_method...",pred,{},True,True,False
mcc,MCC,MCC,<function matthews_corrcoef at 0x0000029497B4D...,"make_scorer(matthews_corrcoef, response_method...",pred,{},True,True,False
balanced_acc,Balance Acc,Balance Acc,<pycaret.internal.metrics.EncodedDecodedLabels...,"make_scorer(balanced_accuracy_score, response_...",pred,{},True,True,True
hamming_loss,Hamming Loss,Hamming Loss,<pycaret.internal.metrics.EncodedDecodedLabels...,"make_scorer(hamming_loss, greater_is_better=Fa...",pred,{},False,True,True


In [44]:
num_iterations_tuning = 20
optimized_metric = 'F1'

## Light Gradient Boosting Machine

In [45]:
lightgbm = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.9600,0.9906,0.9600,0.9608,0.9589,0.8821,0.8861,0.9197,0.0400,0.8306,0.0956
1,0.9609,0.9916,0.9609,0.9617,0.9599,0.8851,0.8889,0.9218,0.0391,0.8347,0.0931
2,0.9610,0.9911,0.9610,0.9617,0.9600,0.8853,0.8890,0.9221,0.0390,0.8350,0.0943
3,0.9621,0.9917,0.9621,0.9628,0.9611,0.8887,0.8922,0.9242,0.0379,0.8395,0.0918
4,0.9633,0.9909,0.9633,0.9639,0.9625,0.8926,0.8958,0.9272,0.0367,0.8449,0.0924
5,0.9597,0.9909,0.9597,0.9606,0.9586,0.8812,0.8854,0.9188,0.0403,0.8293,0.0964
6,0.9601,0.9911,0.9601,0.9609,0.9591,0.8828,0.8866,0.9206,0.0399,0.8316,0.0952
7,0.9600,0.9910,0.9600,0.9609,0.9589,0.8821,0.8863,0.9192,0.0400,0.8305,0.0958
8,0.9611,0.9907,0.9611,0.9621,0.9600,0.8852,0.8894,0.9207,0.0389,0.8346,0.0959


2025/09/26 10:28:34 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [ ]:
lightgbm_tuned_model, lightgbm_tuner = tune_model(lightgbm, search_library = 'optuna', return_tuner=True, n_iter=num_iterations_tuning, optimize=optimized_metric)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,10:28:34
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
evaluate_model(lightgbm)

In [ ]:
lightgbm_top_features = get_feature_importance_df(lightgbm, features_df_training_normalized)
lightgbm_top_features

In [ ]:
predictions_lightgbm = predict_model(lightgbm, data = features_df_testing_normalized)

In [ ]:
predictions_lightgbm

In [ ]:
get_incorrect_predictions(predictions_lightgbm)

In [ ]:
plotPredictionHistograms(predictions_lightgbm, domain, normalization)

## Random Forest Classifier

In [ ]:
rf = create_model('rf')

In [ ]:
rf_tuned_model, rf_tuner = tune_model(rf, search_library = 'optuna', return_tuner=True, n_iter=num_iterations_tuning, optimize=optimized_metric)

In [ ]:
evaluate_model(rf)

In [ ]:
rf_top_features = get_feature_importance_df(rf, features_df_training_normalized)
rf_top_features

In [ ]:
predictions_rf = predict_model(rf, data = features_df_testing_normalized)

In [ ]:
get_incorrect_predictions(predictions_rf)

In [ ]:
plotPredictionHistograms(predictions_rf, domain, normalization)

## SVM

In [ ]:
svm = create_model('svm')

In [ ]:
svm_tuned_model, svm_tuner = tune_model(svm, search_library = 'optuna', return_tuner=True, n_iter=num_iterations_tuning, optimize=optimized_metric)

In [ ]:
print(svm_tuned_model)

In [ ]:
evaluate_model(svm)

In [ ]:
svm_top_features = get_svm_feature_importance_df(svm, features_df_training_normalized)
svm_top_features

In [ ]:
predictions_svm = predict_model(svm, data=features_df_testing_normalized)

In [ ]:
get_incorrect_predictions(predictions_svm)

In [ ]:
plotPredictionHistograms(predictions_svm, domain, normalization)

# Experiment Setup (DL)

## Configure Data

In [ ]:
train, test = train_test_split(features_df_training_normalized, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)
print(f"Train Shape: {train.shape} | Val Shape: {val.shape} | Test Shape: {test.shape}")

In [ ]:
target = "Label"

categorical_cols = [
    col
    for col in features_df_training_normalized.select_dtypes(include=["object","category"]).columns
    if col != target
]

continuous_cols = features_df_training_normalized.select_dtypes(include=["number"]).columns.tolist()

In [ ]:
print("Target:", target)
print("Categorical inputs:", categorical_cols)  
print("Continuous inputs:", continuous_cols)    

In [ ]:
data_config = DataConfig(
    target=[target],
    continuous_cols=continuous_cols,
    categorical_cols=categorical_cols,
)

In [ ]:
available_gpu=1 if torch.cuda.is_available() else 0
print(f"Available GPU: {'Yes' if available_gpu else 'No'}")

In [ ]:
trainer_config = TrainerConfig(
    auto_lr_find=True,
    max_epochs=20,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    batch_size=256,
)

optimizer_config = OptimizerConfig()

experiment_config = ExperimentConfig(
        project_name="TEST",
        run_name="test",
        log_target="tensorboard",
    )

In [ ]:
n_trials = 20

## Tabnet

In [ ]:
def TabNet_Optimization(trial):
    n_d     = trial.suggest_int("n_d", 4, 64)
    n_a     = trial.suggest_int("n_a", 4, 64)
    n_steps = trial.suggest_int("n_steps", 3, 10)
    gamma   = trial.suggest_float("gamma", 1.0, 2.0)
    embedding_dropout = trial.suggest_float("embedding_dropout", 0, 1)
    lr      = trial.suggest_loguniform("learning_rate", 1e-5, 1e-1)
    
    tabnet_config = TabNetModelConfig(
        task="classification",
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        gamma=gamma,
        embedding_dropout=embedding_dropout,
        learning_rate=lr,
        n_independent=2,
        metrics=[
            "auroc",
            "recall",
            "precision",
            "f1_score",
            "cohen_kappa",
            "matthews_corrcoef",
            "hamming_distance",
            "jaccard_index",
        ],
        metrics_prob_input=[
            True,   # auroc
            False,  # recall
            False,  # precision
            False,  # f1_score
            False,  # cohen_kappa
            False,  # matthews_corrcoef
            False,  # hamming_distance
            False,  # jaccard_index
        ],
        metrics_params=[
            {"average": "macro", "num_classes": 2},  # auroc
            {"average": "macro", "num_classes": 2},  # recall
            {"average": "macro", "num_classes": 2},  # precision
            {"average": "macro", "num_classes": 2},  # f1_score
            {"num_classes": 2},                      # cohen_kappa
            {},                                      # matthews_corrcoef
            {},                                      # hamming_distance
            {"average": "macro", "num_classes": 2},  # jaccard_index
        ]
    )
    
    tabnet_model = TabularModel(
        data_config=data_config,
        model_config=tabnet_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
        verbose=True
    )
    
    tabnet_model.fit(train=train, validation=val)

    preds_df = tabnet_model.predict(val)
    y_pred = preds_df["Label_prediction"].to_numpy()
    y_true = val["Label"].to_numpy()
    return f1_score(y_true, y_pred, average="macro")

In [ ]:
tabnet_study = optuna.create_study(direction="maximize")
tabnet_study.optimize(TabNet_Optimization, n_trials=n_trials)

In [ ]:
print("Best params:", tabnet_study.best_params)
print("Best F1 score:", tabnet_study.best_value)

## GANDALF

In [ ]:
def GANDALF_Optimization(trial):
    gflu_stages               = trial.suggest_int("gflu_stages", 1, 10)
    gflu_dropout              = trial.suggest_float("gflu_dropout", 0.0, 0.5)
    gflu_feature_init_sparsity = trial.suggest_float("gflu_feature_init_sparsity", 0.1, 0.9)
    learnable_sparsity        = trial.suggest_categorical("learnable_sparsity", [True, False])
    embedding_dropout         = trial.suggest_float("embedding_dropout", 0.0, 0.5)
    batch_norm_continuous     = trial.suggest_categorical("batch_norm_continuous_input", [True, False])
    learning_rate             = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)

    # 2) build Gandalf config
    gandalf_config = GANDALFConfig(
        task="classification",
        gflu_stages=gflu_stages,
        gflu_dropout=gflu_dropout,
        gflu_feature_init_sparsity=gflu_feature_init_sparsity,
        learnable_sparsity=learnable_sparsity,
        embedding_dropout=embedding_dropout,
        batch_norm_continuous_input=batch_norm_continuous,
        learning_rate=learning_rate,
        metrics=[
            "auroc",
            "recall",
            "precision",
            "f1_score",
            "cohen_kappa",
            "matthews_corrcoef",
            "hamming_distance",
            "jaccard_index",
        ],
        metrics_prob_input=[
            True,   # auroc
            False,  # recall
            False,  # precision
            False,  # f1_score
            False,  # cohen_kappa
            False,  # matthews_corrcoef
            False,  # hamming_distance
            False,  # jaccard_index
        ],
        metrics_params=[
            {"average": "macro", "num_classes": 2},  # auroc
            {"average": "macro", "num_classes": 2},  # recall
            {"average": "macro", "num_classes": 2},  # precision
            {"average": "macro", "num_classes": 2},  # f1_score
            {"num_classes": 2},                      # cohen_kappa
            {},                                      # matthews_corrcoef
            {},                                      # hamming_distance
            {"average": "macro", "num_classes": 2},  # jaccard_index
        ]
    )

    # 3) instantiate & train
    model = TabularModel(
        data_config=data_config,
        model_config=gandalf_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
        verbose=True
    )
    model.fit(train=train, validation=val)

    # 4) predict & return macro-F1
    preds = model.predict(val)
    y_pred = preds["Label_prediction"].to_numpy()
    y_true = val["Label"].to_numpy()
    return f1_score(y_true, y_pred, average="macro")

In [ ]:
gandalf_study = optuna.create_study(direction="maximize")
gandalf_study.optimize(GANDALF_Optimization, n_trials=n_trials)

In [ ]:
print("Best params:", gandalf_study.best_params)
print("Best F1 score:", gandalf_study.best_value)

## FTTransformerModel

In [ ]:
def FTTransformerModel_Optimization(trial):
    input_embed_dim           = trial.suggest_int("input_embed_dim", 32, 128, step=32)
    embedding_initialization  = trial.suggest_categorical("embedding_initialization", ["kaiming_uniform","kaiming_normal"])
    embedding_bias            = trial.suggest_categorical("embedding_bias", [True, False])
    share_embedding           = trial.suggest_categorical("share_embedding", [True, False])
    share_embedding_strategy  = trial.suggest_categorical("share_embedding_strategy", ["add","fraction"])
    shared_embedding_fraction = trial.suggest_float("shared_embedding_fraction", 0.1, 0.5)
    attn_feature_importance   = trial.suggest_categorical("attn_feature_importance", [True, False])
    num_heads                 = num_heads = trial.suggest_categorical("num_heads", [2,4,8,16])
    num_attn_blocks           = trial.suggest_int("num_attn_blocks", 1, 6)
    transformer_head_dim      = trial.suggest_int("transformer_head_dim", 32, 256, step=32)
    attn_dropout              = trial.suggest_float("attn_dropout", 0.0, 0.5)
    add_norm_dropout          = trial.suggest_float("add_norm_dropout", 0.0, 0.5)
    ff_dropout                = trial.suggest_float("ff_dropout", 0.0, 0.5)
    ff_hidden_multiplier      = trial.suggest_int("ff_hidden_multiplier", 1, 8)
    transformer_activation    = trial.suggest_categorical(
        "transformer_activation",
        ["ReLU","LeakyReLU","GEGLU","ReGLU","SwiGLU"]
    )
    embedding_dropout         = trial.suggest_float("embedding_dropout", 0.0, 0.5)
    batch_norm_continuous     = trial.suggest_categorical("batch_norm_continuous_input", [True, False])
    learning_rate             = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)

    # — build FT-Transformer config —
    ft_config = FTTransformerConfig(
        task="classification",
        input_embed_dim=input_embed_dim,
        embedding_initialization=embedding_initialization,
        embedding_bias=embedding_bias,
        share_embedding=share_embedding,
        share_embedding_strategy=share_embedding_strategy,
        shared_embedding_fraction=shared_embedding_fraction,
        attn_feature_importance=attn_feature_importance,
        num_heads=num_heads,
        num_attn_blocks=num_attn_blocks,
        transformer_head_dim=transformer_head_dim,
        attn_dropout=attn_dropout,
        add_norm_dropout=add_norm_dropout,
        ff_dropout=ff_dropout,
        ff_hidden_multiplier=ff_hidden_multiplier,
        transformer_activation=transformer_activation,
        embedding_dropout=embedding_dropout,
        batch_norm_continuous_input=batch_norm_continuous,
        learning_rate=learning_rate,
        metrics=[
            "auroc",
            "recall",
            "precision",
            "f1_score",
            "cohen_kappa",
            "matthews_corrcoef",
            "hamming_distance",
            "jaccard_index",
        ],
        metrics_prob_input=[
            True,   # auroc
            False,  # recall
            False,  # precision
            False,  # f1_score
            False,  # cohen_kappa
            False,  # matthews_corrcoef
            False,  # hamming_distance
            False,  # jaccard_index
        ],
        metrics_params=[
            {"average": "macro", "num_classes": 2},  # auroc
            {"average": "macro", "num_classes": 2},  # recall
            {"average": "macro", "num_classes": 2},  # precision
            {"average": "macro", "num_classes": 2},  # f1_score
            {"num_classes": 2},                      # cohen_kappa
            {},                                      # matthews_corrcoef
            {},                                      # hamming_distance
            {"average": "macro", "num_classes": 2},  # jaccard_index
        ],
    )

    model = TabularModel(
        data_config=data_config,
        model_config=ft_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
        verbose=True,
    )
    model.fit(train=train, validation=val)

    # 4) predict & return macro-F1
    preds = model.predict(val)
    y_pred = preds["Label_prediction"].to_numpy()
    y_true = val["Label"].to_numpy()
    return f1_score(y_true, y_pred, average="macro")

In [ ]:
fft_study = optuna.create_study(direction="maximize")
fft_study.optimize(FTTransformerModel_Optimization, n_trials=n_trials)

In [ ]:
print("Best params:", fft_study.best_params)
print("Best F1 score:", fft_study.best_value)